In [23]:
!pip install fastf1 gradio pandas matplotlib seaborn


In [24]:
import fastf1
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
import time
fastf1.Cache.enable_cache('cache')  # Creates a cache directory for faster data access


In [25]:
def get_all_gps(year):
    schedule = fastf1.get_event_schedule(year)
    return list(schedule["EventName"])


In [30]:
import fastf1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
import os

# ✅ Ensure Cache Directory Exists
if not os.path.exists("cache"):
    os.makedirs("cache")
fastf1.Cache.enable_cache("cache")

# 📊 **Main Race Analysis Function**
def analyze_race(year, gp_name):
    try:
        print(f"🔄 Fetching session for {gp_name} GP {year}...")
        session = fastf1.get_session(year, gp_name, 'R')

        print("📥 Loading session data...")
        session.load()

        if session.laps.empty:
            return "⚠ No lap data available.", None, None, None, None, None, None, None

        print("✅ Data Loaded Successfully!")

        # Extract Data
        laps = session.laps
        weather_data = session.weather_data

        # 🔄 **Extract Tyre Strategy**
        pit_stops = laps[["Driver", "LapNumber", "Compound"]].dropna()

        # 📈 **Lap Time Visualization**
        fig1, ax1 = plt.subplots(figsize=(10, 5))
        for driver in pd.unique(laps['Driver']):
            driver_laps = laps.pick_driver(driver)
            ax1.plot(driver_laps['LapNumber'], driver_laps['LapTime'].dt.total_seconds(), label=driver)
        ax1.set_xlabel('Lap Number')
        ax1.set_ylabel('Lap Time (seconds)')
        ax1.set_title(f'Lap Times - {gp_name} GP {year}')
        ax1.legend()

        # 🌦️ **Weather Data Visualization**
        fig2, ax2 = plt.subplots(figsize=(10, 5))
        ax2.plot(weather_data['Time'], weather_data['AirTemp'], label="Air Temp (°C)", color="red")
        ax2.plot(weather_data['Time'], weather_data['TrackTemp'], label="Track Temp (°C)", color="blue")
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Temperature (°C)')
        ax2.set_title(f'Weather Conditions - {gp_name} GP {year}')
        ax2.legend()

        # 🏎️ **Tyre Strategy Visualization**
        fig3, ax3 = plt.subplots(figsize=(10, 5))
        sns.barplot(x="LapNumber", y="Driver", hue="Compound", data=pit_stops, ax=ax3)
        ax3.set_xlabel("Lap Number")
        ax3.set_ylabel("Driver")
        ax3.set_title(f"Tyre Strategy - {gp_name} GP {year}")

        # 🏁 **Overtakes Per Driver**
        fig4, ax4 = plt.subplots(figsize=(10, 5))
        overtake_counts = laps.groupby('Driver')['LapNumber'].count().sort_values(ascending=False)
        sns.barplot(x=overtake_counts.index, y=overtake_counts.values, ax=ax4)
        ax4.set_xlabel("Driver")
        ax4.set_ylabel("Overtakes")
        ax4.set_title(f"Overtakes per Driver - {gp_name} GP {year}")

        # 🔥 **Fastest Lap per Driver**
        fig5, ax5 = plt.subplots(figsize=(10, 5))
        fastest_laps = laps.loc[laps.groupby('Driver')['LapTime'].idxmin()]
        sns.scatterplot(x=fastest_laps['Driver'], y=fastest_laps['LapTime'].dt.total_seconds(), color='red', ax=ax5)
        ax5.set_xlabel("Driver")
        ax5.set_ylabel("Fastest Lap Time (seconds)")
        ax5.set_title(f"Fastest Lap per Driver - {gp_name} GP {year}")

        # 🏎️ **Race Pace Comparison (Rolling Avg Lap Time)**
        fig6, ax6 = plt.subplots(figsize=(10, 5))
        for driver in pd.unique(laps['Driver']):
            driver_laps = laps.pick_driver(driver)
            ax6.plot(driver_laps['LapNumber'], driver_laps['LapTime'].dt.total_seconds().rolling(5).mean(), label=driver)
        ax6.set_xlabel('Lap Number')
        ax6.set_ylabel('Rolling Avg Lap Time (5 Laps)')
        ax6.set_title(f'Race Pace Comparison - {gp_name} GP {year}')
        ax6.legend()

        # 📉 **Tyre Degradation Trends**
        fig7, ax7 = plt.subplots(figsize=(10, 5))
        for driver in pd.unique(laps['Driver']):
            driver_laps = laps.pick_driver(driver)
            ax7.plot(driver_laps['LapNumber'], driver_laps['LapTime'].dt.total_seconds(), label=driver)
        ax7.set_xlabel('Lap Number')
        ax7.set_ylabel('Lap Time (seconds)')
        ax7.set_title(f'Tyre Degradation Trends - {gp_name} GP {year}')
        ax7.legend()

        return fig1, fig2, fig3, fig4, fig5, fig6, fig7

    except Exception as e:
        return f"❌ Error: {str(e)}", None, None, None, None, None, None, None


# 🚀 **Gradio UI for User Input**
def gradio_interface(year, gp_name):
    return analyze_race(int(year), gp_name)

# ✅ Available Grand Prix Races
grand_prix_options = [
    "Bahrain", "Saudi Arabia", "Australia", "Azerbaijan", "Miami",
    "Imola", "Monaco", "Spain", "Canada", "Austria",
    "Britain", "Hungary", "Belgium", "Netherlands", "Italy",
    "Singapore", "Japan", "Qatar", "USA", "Mexico", "Brazil",
    "Las Vegas", "Abu Dhabi"
]

# 🎨 **Create Gradio App**
iface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Dropdown(choices=[str(y) for y in range(2018, 2025)], label="Year"),
        gr.Dropdown(choices=grand_prix_options, label="Grand Prix"),
    ],
    outputs=[
        gr.Plot(label="Lap Times"),
        gr.Plot(label="Weather Conditions"),
        gr.Plot(label="Tyre Strategy"),
        gr.Plot(label="Overtakes per Driver"),
        gr.Plot(label="Fastest Lap per Driver"),
        gr.Plot(label="Race Pace Comparison"),
        gr.Plot(label="Tyre Degradation Trends"),
    ],
    title="🏎️ F1 Race Strategy & Analysis",
    description="Select a race year and Grand Prix to visualize strategy insights."
)

# 🌐 **Launch Gradio Web App**
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9f4c70410085a82591.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
